# Generating Adversarial Domain Names with MaskDGA-WB
This notebook contains example code for generating fully valid adversarial e2LDs using the modified white-box part of the MaskDGA adversarial attack.

MaskDGA was introduced in the paper:

Lior Sidi, Asaf Nadler, and Asaf Shabtai. “MaskDGA: An Evasion Attack
Against DGA Classifiers and Adversarial Defenses”. In: IEEE Access 8 (2020),
pp. 161580–161592. doi: 10.1109/ACCESS.2020.3020964.

The concepts are explained step by step, allowing you to apply these attacks to your own models!

In [1]:
import torch
import os
from robust_dga_detection.models import CNNResNetWithEmbedding
from robust_dga_detection.utils import domains, reproduceability

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
reproduceability.setup_deterministic_environment()

## Step 1: Load a model to attack

In [2]:
model = CNNResNetWithEmbedding().to(DEVICE)

In [3]:
model.load_state_dict(
    torch.load(os.getenv("TRAINED_MODEL_PATH"))
)

<All keys matched successfully>

## Step 2: Setup the MaskDGA-WB Attack

Our implementation of the MaskDGA-WB attack internally uses a One-Hot representation of domain names for more convinient gradient computation.
We use the `OneHotModelInputWrapper` to facilitate a model that can accept such inputs.

In [4]:
from robust_dga_detection.attacks.nlp import OneHotModelInputWrapper, MaskDGA

onehot_model = OneHotModelInputWrapper(model)
attack = MaskDGA(onehot_model)

## Step 3: Generate adversarial domain names

In [5]:
input_domain = "lmlabssssssssentasdasdasdasdasdasdasd"

Measure the models prediction on the input domain

In [7]:
encoded_domains = torch.unsqueeze(domains.encode_domain(input_domain), dim=0).to(DEVICE)
labels = torch.Tensor([1]).long().to(DEVICE)
print(f"Baseline model value: {torch.sigmoid(model(encoded_domains)).item()}")

Baseline model value: 0.9897634983062744


Use MaskDGA-WB to generate an adversarial domain and measure how good the generated AE is

In [8]:
adversarial_encoded_domains = attack([input_domain], encoded_domains, labels)
generated_domain = domains.decode_domains(adversarial_encoded_domains)[0]

print(f"Generated adversarial domain {generated_domain}")
print(f"Model-Value on adversarial domain {torch.sigmoid(model(adversarial_encoded_domains)).item()}")

Generated adversarial domain lmla-ssssss-se-ta-d0-90--0-01-d5s11sd
Model-Value on adversarial domain 6.23141739417613e-25


We hope this notebook provided you with the details required for testing your models against our attacks.
Nevertheless, do not hesitate to reach out if you have any questions.